# A1111 Extension Installer

Installs **Automatic1111** extensions into the WebUI **data directory** so they are picked up correctly:

- Data root (WEBUI_ROOT): `/workspace/a1111` (default in your image)
- Extensions dir: `/workspace/a1111/extensions`

**How it works**
- **Defaults**: always installed (safe, lightweight helpers)
- **Recommended**: toggle `enable: True/False` per extension
- **Extra URLs**: paste any Git repo URLs to install as well

**Notes**
- Running again is safe: existing repos are updated with `git pull`.
- After installing, use the **Restart WebUI** cell below (or run the command shown) so A1111 loads the new extensions.

In [ ]:
# --- Paths / settings ---
import os
from pathlib import Path

# Where A1111 expects user data (models, embeddings, *extensions*, etc.)
WEBUI_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
EXT_DIR = WEBUI_ROOT / "extensions"
EXT_DIR.mkdir(parents=True, exist_ok=True)

print("WEBUI_ROOT:", WEBUI_ROOT)
print("Extensions directory:", EXT_DIR)

# --- Defaults: always install ---
DEFAULT_EXTS = [
    {"name": "Aspect Ratio",       "url": "https://github.com/alemelis/sd-webui-ar"},
    {"name": "Tag Autocomplete",   "url": "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"},
    {"name": "Images Browser",     "url": "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"},
    {"name": "Config Presets",     "url": "https://github.com/Zyin055/Config-Presets"}
]

# --- Recommended: toggle enable True/False ---
RECOMMENDED = [
    {"name": "ControlNet",            "url": "https://github.com/Mikubill/sd-webui-controlnet",                 "enable": False},
    {"name": "ADetailer",             "url": "https://github.com/Bing-su/adetailer",                            "enable": True},
    {"name": "Ultimate SD Upscale",   "url": "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111", "enable": True},
    {"name": "Dynamic Prompts",       "url": "https://github.com/adieyal/sd-dynamic-prompts",                   "enable": False},
    {"name": "Openpose Editor",       "url": "https://github.com/fkunn1326/openpose-editor",                    "enable": False}
]

# --- Extra URLs: paste any additional Git repos here ---
EXTRA_URLS = [
    # e.g. "https://github.com/someuser/some-extension"
]

print("Configured:", len(DEFAULT_EXTS), "defaults,",
      sum(1 for r in RECOMMENDED if r.get("enable")), "recommended enabled,",
      len(EXTRA_URLS), "extra URLs")

In [ ]:
import subprocess, re
from pathlib import Path
from urllib.parse import urlparse

def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_basename(url: str) -> str:
    # turn https://github.com/owner/repo(.git) into "repo"
    path = urlparse(url).path.rstrip("/")
    name = path.split("/")[-1]
    return name[:-4] if name.endswith(".git") else name

def install_or_update(url: str, base: Path) -> Path:
    name = repo_basename(url)
    dst = base / name
    if not dst.exists():
        print(f"\n[install] {name} from {url}")
        run(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {name}")
        try:
            run(["git", "-C", str(dst), "reset", "--hard"]) 
            run(["git", "-C", str(dst), "pull", "--ff-only"]) 
        except subprocess.CalledProcessError:
            print("  (non-ff; attempting rebase)")
            run(["git", "-C", str(dst), "pull", "--rebase", "--autostash"]) 
    return dst

In [ ]:
installed, errors = [], []

# 1) defaults
for ext in DEFAULT_EXTS:
    try:
        installed.append(install_or_update(ext["url"], EXT_DIR))
    except Exception as e:
        errors.append((ext["name"], str(e)))

# 2) recommended (enabled only)
for rec in RECOMMENDED:
    if rec.get("enable"):
        try:
            installed.append(install_or_update(rec["url"], EXT_DIR))
        except Exception as e:
            errors.append((rec["name"], str(e)))

# 3) extras
for url in EXTRA_URLS:
    try:
        installed.append(install_or_update(url, EXT_DIR))
    except Exception as e:
        errors.append((url, str(e)))

print("\n=== Summary ===")
for p in installed:
    print("✔", p)
if errors:
    print("\nErrors:")
    for name, msg in errors:
        print("✖", name, "→", msg)

## Restart WebUI (required after installing)
This restarts A1111 under **supervisord**. If the supervisor control socket is unavailable, it falls back to killing `launch.py` so Supervisor auto-restarts it.

In [ ]:
import os, time, subprocess, shlex

print("Attempting to restart the A1111 WebUI service via supervisord …")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print((out.stdout or out.stderr).strip() or "✔ Restarted via supervisorctl")
    else:
        print("Supervisorctl socket unavailable, forcing process restart…")
        subprocess.run("pkill -f 'launch.py'", shell=True)
        time.sleep(5)
        print("✔ WebUI process terminated, Supervisor will auto-restart it")
    print("Wait ~15 seconds, then reload your WebUI tab.")
except Exception as e:
    print("⚠️ Restart attempt failed:", e)
    print("If needed, run this in a Terminal:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")

## Next steps
- Refresh the WebUI (port `7860`).
- Extensions should now appear under **Extensions → Installed**.
- If an extension requires models, place them under:
  - ControlNet: `/workspace/a1111/models/ControlNet/*.safetensors|*.pth`
  - AnimateDiff: `/workspace/a1111/models/AnimateDiff/*.safetensors`
  - ReActor (InsightFace):
      - `/workspace/a1111/models/insightface/models/`
      - `/workspace/a1111/models/insightface/antelopev2/`